In [1]:
%pip install numpy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import random

In [3]:

#Base class, any child classes should implement evolve and have a value
class Asset:
    def __init__(self, id, initialValue):
        self.value = initialValue
        self.initialValue = initialValue
        self.assetId = id
        self.bankrupt = False
    def evolve(self):
        self.value = self.value

    def reset(self):
        self.value = self.initialValue

class Bond(Asset):
    def __init__(self, id, initialValue, rate):
        self.rate = rate
        super().__init__(id, initialValue)
    def evolve(self):
        self.value = self.value * self.rate

#Modified so that a price of <=0 is an absorbing state
class BinomialStock(Asset):
    def __init__(self, id, initialValue, u, d, p):
        self.u = u
        self.d = d
        self.p = p
        super().__init__(id, initialValue)

    def evolve(self):
        if not self.bankrupt:
            probDraw = random.random()
            if probDraw < self.p:
                self.value *= self.u
            else:
                self.value += self.d
            if self.value<= 0:
                self.bankrupt = True
                self.value = 0
class MultinomialStock(Asset):
    def __init__(self, id, initialValue, values, probabilities):
        
        self.values = values
        self.probabilities = probabilities
        super().__init__(id, initialValue)

    def evolve(self):
        if not self.bankrupt:
            prob_draw = random.random()
            cumulative_probability = 0

            # Iterate through values and probabilities
            for i, prob in enumerate(self.probabilities):
                cumulative_probability += prob

                # Check if the random draw falls within this probability range
                if prob_draw < cumulative_probability:
                    self.value *= self.values[i]
                    break
            if self.value<= 0:
                self.bankrupt = True
                self.value = 0
class Market:
    def __init__(self, assets):
        #For an asset to be
        self.assets = assets
        self.time = 0
        self.history = []
        self.history.append(self.returnAssetValues())

    def returnAssetValues(self):
        values = []
        for asset in self.assets:
            values.append(asset.value)
        return values

    def evolve(self):
        for asset in self.assets:
            asset.evolve()
        self.history.append(self.returnAssetValues())
        self.time += 1

    def reset(self):
        self.time = 0
        self.history = []
        for asset in self.assets:
            asset.reset()
        self.history.append(self.returnAssetValues())

    def returnHistoricReturns(self):
        marketReturns = []
        #Guranteed to exist by initialistion method
        initialMarketValue = sum(self.history[0])
        for assetPrices in self.history:
            marketReturns.append(sum(assetPrices) - initialMarketValue)
        return marketReturns

    #Beta is the correlation between an asset's returns and the returns of the market as a whole
    def calculateHistoricBeta(self, assetId):
        #Could maybe use a dictionary?
        assetNum = -1
        for i in range(len(self.assets)):
            if self.assets[i].assetId == assetId:
                assetNum = i
        if assetNum == -1:
            raise ValueError("No assets with assetId " + str(assetId)+" found")
        
        marketReturns = self.returnHistoricReturns()
        stockReturns = np.array(self.history)[:,assetNum] - self.assets[i].initialValue
        #Covariance Matrix has form [var(stock), cov(stock, market)][cov(market,stock), var(market)]
        beta = np.cov(stockReturns,marketReturns)[1,0]/np.var(marketReturns)

        return beta

class Portfolio:

    tradingStrategy = None
    market = None

    currentAllocation = []

    def __init__(self, tradingStrategy, market, initalAllocation):
        self.tradingStrategy = tradingStrategy
        self.market = market
        self.currentAllocation = initalAllocation
        self.initialAllocation = initalAllocation
    def stepTime(self):
        self.evolve()
        self.rebalance()

    def rebalance(self):
        newAllocation = self.tradingStrategy(self.market.history)
        currentAssetValues = self.market.history[len(self.market.history)-1]

        #May need to introduce tolerance here
        if (np.dot(self.currentAllocation, currentAssetValues) != np.dot(newAllocation, currentAssetValues)):
            raise ValueError("Value after rebalancing portofolio changed")
        else:
            self.currentAllocation = newAllocation

    def evolve(self):
        self.market.evolve()

    def returnCurrentValue(self):
        currentAssetValues = self.market.history[len(self.market.history)-1]

        return np.dot(currentAssetValues, self.currentAllocation)
    
    def reset(self):
        self.currentAllocation = self.initialAllocation
        self.market.reset()
    